
# Lab 1 — Google Colab Setup for LoRA + Unsloth (SLM Dragon Trainer)

This notebook prepares a clean, reliable Colab environment for fine‑tuning Small Language Models (SLMs) with **LoRA + Unsloth** on a GPU runtime.

**What this does**
- Verifies GPU
- Installs a *matching* PyTorch trio (torch, torchvision, torchaudio) with auto‑fallback across CUDA wheels
- Installs Unsloth + Unsloth Zoo and other LLM libs **without** breaking Torch deps
- Loads a base model and runs a quick inference smoke test

> If a later cell or library upgrades Torch and breaks compatibility, just re‑run the **Install PyTorch** and **Install LLM libs** cells.



## Step 0 — Enable GPU in Colab
- Runtime → **Change runtime type** → Hardware accelerator: **GPU** → *Save*


## Step 1 — Verify GPU

In [1]:

!nvidia-smi || echo "No GPU detected. Enable GPU under Runtime → Change runtime type."


Mon Aug 11 13:57:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             45W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----


## Step 2 — Install a matched PyTorch build (auto‑fallback)
This cell:
- Removes conflicting stacks
- Tries official wheels in order: **cu124 → cu121 → cu118**
- Verifies import and CUDA availability


In [2]:

# Clean potential conflicts
!pip -q uninstall -y torch torchvision torchaudio fastai
!pip -q cache purge

def try_tag(tag):
    print(f"\nTrying {tag} wheels...")
    # Use the official index for speed and correctness
    rc = !pip install --no-cache-dir --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/{tag}
    try:
        import torch
        print("Torch:", torch.__version__, "| CUDA available:", torch.cuda.is_available())
        return True
    except Exception as e:
        print("Torch import failed:", e)
        return False

ok = False
for tag in ["cu124", "cu121", "cu118"]:
    if try_tag(tag):
        ok = True
        break

if not ok:
    raise SystemExit("Could not install a matching torch build. Make sure GPU is enabled, then rerun this cell.")



Trying cu124 wheels...
Torch: 2.6.0+cu124 | CUDA available: True



## Step 3 — Install LLM libraries (without touching Torch)
We use `--no-deps` to prevent accidental Torch upgrades by pip.


In [6]:

# Step 3 — Install LLM libraries (without touching Torch)
!pip install -U unsloth unsloth_zoo accelerate transformers peft datasets bitsandbytes sentencepiece trl --no-deps


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 31.5 MB/s eta 0:00:00


## Step 4 — Sanity check Torch + GPU

In [7]:

import torch
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


Torch: 2.6.0+cu124
CUDA available: True
Device: NVIDIA A100-SXM4-40GB



## Step 5 — Load a base model (Unsloth)
> If you hit permission errors with Llama‑2, switch to the permissive Mistral model line below.


In [10]:

from unsloth import FastLanguageModel
import torch

model_name = "unsloth/mistral-7b-v0.2"  # public, Unsloth-hosted
# Source exists: https://huggingface.co/unsloth/mistral-7b-v0.2

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=2048,
    dtype=torch.float16,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)
print("Loaded:", model_name)



==((====))==  Unsloth 2025.8.4: Fast Mistral patching. Transformers: 4.55.0.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


<string>:37: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.


Loaded: unsloth/mistral-7b-v0.2


## Step 6 — Quick inference smoke test

In [12]:

prompt = "What is the capital of France?"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=32)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


What is the capital of France?

Paris is the capital of France.

What is the capital of France?

Paris is the capital of France.

What



---

## Troubleshooting

- **No GPU detected**: Enable GPU (Step 0) and **Runtime → Restart runtime**, then re‑run from Step 1.
- **Dependency conflicts**: If a later install upgrades Torch, re‑run **Step 2** then **Step 3**.
- **`ImportError: install unsloth_zoo`**: Re‑run **Step 3** to install `unsloth_zoo`.
- **`pip` resolver warnings**: These are expected in Colab’s mixed environment. We explicitly pin Torch trio in Step 2 and use `--no-deps` in Step 3 to avoid breakage.
